# Heart Disease Classification - PostgreSQL Database Setup

This notebook:
1. Downloads the Heart Disease dataset from Kaggle using kagglehub
2. Connects to Render PostgreSQL database
3. Creates a normalized (3NF) database schema
4. Populates the database tables

**Dataset:** Heart Disease Dataset from Kaggle (johnsmith88/heart-disease-dataset)
- 1,025 patients
- 13 features + 1 target
- Binary classification: 0 = No heart disease, 1 = Heart disease

**Database:** PostgreSQL on Render (Cloud)

In [1]:
import os
import pandas as pd
import numpy as np
import json
import kagglehub
import psycopg2
from psycopg2.extras import execute_values
from pathlib import Path
from dotenv import load_dotenv

# Set base folder
base_folder = Path(os.getcwd()).parent
print(f"Base folder: {base_folder}")

# Load environment variables
load_dotenv(base_folder / ".env")
DATABASE_URL = os.getenv("DATABASE_URL")

if not DATABASE_URL:
    raise ValueError("DATABASE_URL not found in .env file")
    
print(f"✓ Database URL configured")

Base folder: /Users/kusumareddy/python_final
✓ Database URL configured


## Step 1: Download Heart Disease Dataset from Kaggle

Using kagglehub to download the official dataset:
- **Dataset ID**: johnsmith88/heart-disease-dataset
- **Features**: age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal
- **Target**: target (0 = no disease, 1 = disease)

In [2]:
# Download Heart Disease dataset from Kaggle using kagglehub
print("📥 Downloading Heart Disease dataset from Kaggle...")
dataset_path = kagglehub.dataset_download("johnsmith88/heart-disease-dataset")
print(f"✓ Dataset downloaded to: {dataset_path}")

# Find the CSV file in the downloaded dataset
dataset_dir = Path(dataset_path)
csv_files = list(dataset_dir.glob("*.csv"))
print(f"Found CSV files: {csv_files}")

# Load the heart disease dataset
if csv_files:
    heart_csv_path = csv_files[0]  # Use the first CSV file found
    print(f"Loading data from: {heart_csv_path}")
    heart_df = pd.read_csv(heart_csv_path)
    
    # Save locally for reference
    local_data_dir = base_folder / "data"
    local_data_dir.mkdir(exist_ok=True)
    local_csv_path = local_data_dir / "heart.csv"
    heart_df.to_csv(local_csv_path, index=False)
    print(f"Dataset saved locally to: {local_csv_path}")
else:
    raise FileNotFoundError("No CSV files found in the downloaded dataset")

print(f"\nDataset shape: {heart_df.shape}")
print(f"Columns: {list(heart_df.columns)}")
heart_df.head()

📥 Downloading Heart Disease dataset from Kaggle...
✓ Dataset downloaded to: /Users/kusumareddy/.cache/kagglehub/datasets/johnsmith88/heart-disease-dataset/versions/2
Found CSV files: [PosixPath('/Users/kusumareddy/.cache/kagglehub/datasets/johnsmith88/heart-disease-dataset/versions/2/heart.csv')]
Loading data from: /Users/kusumareddy/.cache/kagglehub/datasets/johnsmith88/heart-disease-dataset/versions/2/heart.csv
Dataset saved locally to: /Users/kusumareddy/python_final/data/heart.csv

Dataset shape: (1025, 14)
Columns: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [3]:
# Check data info and missing values
print("=" * 50)
print("Data Info:")
print("=" * 50)
print(heart_df.info())
print("\n" + "=" * 50)
print("Missing Values:")
print("=" * 50)
print(heart_df.isnull().sum())
print("\n" + "=" * 50)
print("Target Distribution:")
print("=" * 50)
print(heart_df['target'].value_counts())
print(f"\nClass balance: {heart_df['target'].value_counts(normalize=True).round(3)}")

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB
None

Missing Values:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
t

## Step 2: Connect to Render PostgreSQL Database

In [4]:
# Connect to Render PostgreSQL
print("🔌 Connecting to Render PostgreSQL...")

# Add SSL configuration for Render
if "?sslmode=" not in DATABASE_URL:
    database_url_with_ssl = DATABASE_URL + "?sslmode=require"
else:
    database_url_with_ssl = DATABASE_URL

try:
    conn = psycopg2.connect(database_url_with_ssl)
    cursor = conn.cursor()
    
    # Test connection
    cursor.execute("SELECT version()")
    version = cursor.fetchone()[0]
    
    cursor.execute("SELECT current_database()")
    db_name = cursor.fetchone()[0]
    
    print(f"✅ Connected to PostgreSQL!")
    print(f"   Database: {db_name}")
    print(f"   Version: {version}")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise

🔌 Connecting to Render PostgreSQL...
✅ Connected to PostgreSQL!
   Database: heart_disease_8qhr
   Version: PostgreSQL 18.1 (Debian 18.1-1.pgdg12+2) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit


## Step 3: Create 3NF Normalized Database Schema

We'll normalize the data into three main tables:
1. **patient_demographics** - Patient ID, age, sex
2. **patient_clinical** - Clinical measurements and test results
3. **patient_outcomes** - Heart disease diagnosis (target)

Plus lookup tables for categorical values to ensure referential integrity.

In [5]:
def create_postgresql_schema(conn):
    """Create 3NF normalized schema in PostgreSQL"""
    print("🏗️  Creating PostgreSQL schema...")
    
    cursor = conn.cursor()
    
    # Drop existing tables (in correct order due to foreign keys)
    cursor.execute("""
        DROP TABLE IF EXISTS patient_outcomes CASCADE;
        DROP TABLE IF EXISTS patient_clinical CASCADE;
        DROP TABLE IF EXISTS patient_demographics CASCADE;
        DROP TABLE IF EXISTS chest_pain_types CASCADE;
        DROP TABLE IF EXISTS thal_types CASCADE;
        DROP TABLE IF EXISTS restecg_types CASCADE;
        DROP TABLE IF EXISTS slope_types CASCADE;
        DROP VIEW IF EXISTS patient_ml_data CASCADE;
    """)
    
    # Create lookup tables
    cursor.execute("""
        CREATE TABLE chest_pain_types (
            cp_id INTEGER PRIMARY KEY,
            cp_name TEXT NOT NULL UNIQUE
        );
        
        CREATE TABLE thal_types (
            thal_id INTEGER PRIMARY KEY,
            thal_name TEXT NOT NULL UNIQUE
        );
        
        CREATE TABLE restecg_types (
            restecg_id INTEGER PRIMARY KEY,
            restecg_name TEXT NOT NULL UNIQUE
        );
        
        CREATE TABLE slope_types (
            slope_id INTEGER PRIMARY KEY,
            slope_name TEXT NOT NULL UNIQUE
        );
    """)
    
    # Create main tables with proper constraints
    cursor.execute("""
        CREATE TABLE patient_demographics (
            patient_id INTEGER PRIMARY KEY,
            age INTEGER NOT NULL CHECK (age > 0 AND age < 150),
            sex INTEGER NOT NULL CHECK (sex IN (0, 1))
        );
        
        CREATE TABLE patient_clinical (
            patient_id INTEGER PRIMARY KEY,
            cp INTEGER NOT NULL CHECK (cp IN (0, 1, 2, 3)),
            trestbps INTEGER NOT NULL CHECK (trestbps > 0),
            chol INTEGER NOT NULL CHECK (chol > 0),
            fbs INTEGER NOT NULL CHECK (fbs IN (0, 1)),
            restecg INTEGER NOT NULL CHECK (restecg IN (0, 1, 2)),
            thalach INTEGER NOT NULL CHECK (thalach > 0),
            exang INTEGER NOT NULL CHECK (exang IN (0, 1)),
            oldpeak REAL NOT NULL CHECK (oldpeak >= 0),
            slope INTEGER NOT NULL CHECK (slope IN (0, 1, 2)),
            ca INTEGER NOT NULL CHECK (ca >= 0 AND ca <= 4),
            thal INTEGER NOT NULL CHECK (thal IN (0, 1, 2, 3)),
            FOREIGN KEY (patient_id) REFERENCES patient_demographics(patient_id)
        );
        
        CREATE TABLE patient_outcomes (
            patient_id INTEGER PRIMARY KEY,
            target INTEGER NOT NULL CHECK (target IN (0, 1)),
            FOREIGN KEY (patient_id) REFERENCES patient_demographics(patient_id)
        );
    """)
    
    # Create indexes for performance
    cursor.execute("""
        CREATE INDEX idx_demographics_age ON patient_demographics(age);
        CREATE INDEX idx_demographics_sex ON patient_demographics(sex);
        CREATE INDEX idx_clinical_cp ON patient_clinical(cp);
        CREATE INDEX idx_outcomes_target ON patient_outcomes(target);
    """)
    
    # Insert lookup data
    lookup_data = [
        ("chest_pain_types", [(0, 'Typical Angina'), (1, 'Atypical Angina'), 
                             (2, 'Non-anginal Pain'), (3, 'Asymptomatic')]),
        ("thal_types", [(0, 'Normal'), (1, 'Fixed Defect'), 
                       (2, 'Reversible Defect'), (3, 'Unknown')]),
        ("restecg_types", [(0, 'Normal'), (1, 'ST-T Wave Abnormality'), 
                          (2, 'Left Ventricular Hypertrophy')]),
        ("slope_types", [(0, 'Upsloping'), (1, 'Flat'), (2, 'Downsloping')])
    ]
    
    for table_name, data in lookup_data:
        execute_values(
            cursor,
            f"INSERT INTO {table_name} VALUES %s",
            data,
            template=None,
            page_size=100
        )
    
    # Create ML data view
    cursor.execute("""
        CREATE VIEW patient_ml_data AS
        SELECT
            d.patient_id, d.age, d.sex,
            c.cp, c.trestbps, c.chol, c.fbs, c.restecg,
            c.thalach, c.exang, c.oldpeak, c.slope, c.ca, c.thal,
            o.target
        FROM patient_demographics d
        JOIN patient_clinical c ON d.patient_id = c.patient_id
        JOIN patient_outcomes o ON d.patient_id = o.patient_id
        ORDER BY d.patient_id;
    """)
    
    conn.commit()
    print("✅ PostgreSQL schema created successfully")

# Create the schema
create_postgresql_schema(conn)

🏗️  Creating PostgreSQL schema...
✅ PostgreSQL schema created successfully


## Step 4: Populate Database with Heart Disease Data

In [6]:
def migrate_data_to_postgresql(heart_df, conn):
    """Migrate heart disease data to PostgreSQL"""
    print("📊 Migrating data to PostgreSQL...")
    
    # Add patient_id
    heart_df = heart_df.reset_index().rename(columns={"index": "patient_id"})
    
    cursor = conn.cursor()
    
    # Prepare data for insertion
    demographics_data = heart_df[['patient_id', 'age', 'sex']].values.tolist()
    clinical_data = heart_df[[
        'patient_id', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
        'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'
    ]].values.tolist()
    outcomes_data = heart_df[['patient_id', 'target']].values.tolist()
    
    # Insert data
    execute_values(
        cursor,
        "INSERT INTO patient_demographics (patient_id, age, sex) VALUES %s",
        demographics_data,
        template=None,
        page_size=100
    )
    
    execute_values(
        cursor,
        """INSERT INTO patient_clinical 
           (patient_id, cp, trestbps, chol, fbs, restecg, thalach, 
            exang, oldpeak, slope, ca, thal) VALUES %s""",
        clinical_data,
        template=None,
        page_size=100
    )
    
    execute_values(
        cursor,
        "INSERT INTO patient_outcomes (patient_id, target) VALUES %s",
        outcomes_data,
        template=None,
        page_size=100
    )
    
    conn.commit()
    print(f"✅ Migrated {len(heart_df)} patient records to PostgreSQL")
    
    return heart_df

# Migrate the data
heart_data = migrate_data_to_postgresql(heart_df, conn)

📊 Migrating data to PostgreSQL...
✅ Migrated 1025 patient records to PostgreSQL


## Step 5: Verify Database Setup

In [7]:
# Verify database structure and data integrity
print("🔍 Verifying database setup...")

cursor = conn.cursor()

# Check table counts
tables = ['patient_demographics', 'patient_clinical', 'patient_outcomes',
          'chest_pain_types', 'thal_types', 'restecg_types', 'slope_types']

print("\nTable row counts:")
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"  {table}: {count} rows")

# Test the ML data view
print("\n📊 Testing ML data view...")
cursor.execute("SELECT * FROM patient_ml_data LIMIT 5")
sample_data = cursor.fetchall()

print(f"Sample data from patient_ml_data view ({len(sample_data)} rows):")
for row in sample_data:
    print(f"  Patient {row[0]}: age={row[1]}, sex={row[2]}, target={row[-1]}")

# Check target distribution
cursor.execute("SELECT target, COUNT(*) FROM patient_outcomes GROUP BY target ORDER BY target")
target_dist = cursor.fetchall()
print(f"\n✅ Target distribution: {dict(target_dist)}")

# Test loading data as DataFrame
print("\n📈 Testing DataFrame loading...")
test_df = pd.read_sql_query("SELECT * FROM patient_ml_data", conn)
print(f"✅ Loaded {len(test_df)} rows as DataFrame")
print(f"   Columns: {list(test_df.columns)}")

🔍 Verifying database setup...

Table row counts:
  patient_demographics: 1025 rows
  patient_clinical: 1025 rows
  patient_outcomes: 1025 rows
  chest_pain_types: 4 rows
  thal_types: 4 rows
  restecg_types: 3 rows
  slope_types: 3 rows

📊 Testing ML data view...
Sample data from patient_ml_data view (5 rows):
  Patient 0: age=52, sex=1, target=0
  Patient 1: age=53, sex=1, target=0
  Patient 2: age=70, sex=1, target=0
  Patient 3: age=61, sex=1, target=0
  Patient 4: age=62, sex=0, target=0

✅ Target distribution: {0: 499, 1: 526}

📈 Testing DataFrame loading...


/var/folders/r3/xktxz7_x47572mmrj_j_wtn40000gn/T/ipykernel_48294/722120517.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  test_df = pd.read_sql_query("SELECT * FROM patient_ml_data", conn)


✅ Loaded 1025 rows as DataFrame
   Columns: ['patient_id', 'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']


## Step 6: Generate Data Schema for Streamlit

In [10]:
# Generate data schema JSON for Streamlit UI
print("📋 Generating data schema for Streamlit...")

numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

schema = {"numerical": {}, "categorical": {}}

# Numerical features statistics
for feat in numerical_features:
    schema["numerical"][feat] = {
        "min": float(test_df[feat].min()),
        "max": float(test_df[feat].max()),
        "mean": float(test_df[feat].mean()),
        "median": float(test_df[feat].median()),
        "std": float(test_df[feat].std())
    }

# Save schema
schema_path = base_folder / "data" / "data_schema.json"
with open(schema_path, 'w') as f:
    json.dump(schema, f, indent=2)

print(f"✅ Schema saved to {schema_path}")

📋 Generating data schema for Streamlit...
✅ Schema saved to /Users/kusumareddy/python_final/data/data_schema.json


In [9]:
# Close database connection
conn.close()

print("\n" + "=" * 70)
print("🎉 POSTGRESQL DATABASE SETUP COMPLETE!")
print("=" * 70)
print("✅ Dataset downloaded from Kaggle using kagglehub")
print("✅ Connected to Render PostgreSQL database")
print("✅ 3NF normalized schema created")
print("✅ All data migrated successfully")
print("✅ ML data view created for training")
print("✅ Data schema generated for Streamlit UI")
print(f"✅ Total patients: {len(test_df)}")
print(f"✅ Target distribution: No Disease={sum(test_df['target']==0)}, Disease={sum(test_df['target']==1)}")
print(f"✅ Class balance: {test_df['target'].value_counts(normalize=True).round(3).to_dict()}")
print("\n🚀 Ready for ML experiments and deployment!")


🎉 POSTGRESQL DATABASE SETUP COMPLETE!
✅ Dataset downloaded from Kaggle using kagglehub
✅ Connected to Render PostgreSQL database
✅ 3NF normalized schema created
✅ All data migrated successfully
✅ ML data view created for training
✅ Data schema generated for Streamlit UI
✅ Total patients: 1025
✅ Target distribution: No Disease=499, Disease=526
✅ Class balance: {1: 0.513, 0: 0.487}

🚀 Ready for ML experiments and deployment!
